In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

Возьмем датасет с кредитным скорингом

In [2]:
path = '/home/maxim/Документы/GeekBrains/4 четверть. Машинное обучение в бизнесе/Manuals/lection6_materials/SouthGermanCredit.asc'
df = pd.read_csv(path, sep=' ')
df.head(5)

,status,duration,credit_history,purpose,amount,savings,employment_duration,installment_rate,personal_status_sex,other_debtors,...,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker,credit_risk
0,1,18,4,2,1049,1,2,4,2,1,...,2,21,3,1,1,3,2,1,2,1
1,1,9,4,0,2799,1,3,2,3,1,...,1,36,3,1,2,3,1,1,2,1
2,2,12,2,9,841,2,4,2,2,1,...,1,23,3,1,1,2,2,1,2,1
3,1,12,4,0,2122,1,3,3,3,1,...,1,39,3,1,2,2,1,1,1,1
4,1,12,4,0,2171,1,3,4,3,1,...,2,38,1,2,2,2,2,1,1,1


Добавим одну бинарную фичу - плановая дата погашения кредита до наступления пенсионного возраста (0) или после (1)

In [3]:
df.insert(20, 'return_after_ret_age', 0)
df.loc[(((df['duration'] / 12) + df['age']) > 65), 'return_after_ret_age'] = 1

Разделим датасет на тренировочную и тестовую выборки

In [4]:
x_data = df.iloc[:,:-1]
y_data = df.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=42)

Построим модель на основе классификатора XGBoost

In [5]:
model = XGBClassifier(use_label_encoder=False)

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

[17:47:55] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Вынесем соответствующие метрики в функцию

In [6]:
def evaluate_results(y_test, y_predict):
    
    f1 = f1_score(y_test, y_predict)
    roc = roc_auc_score(y_test, y_predict)
    rec = recall_score(y_test, y_predict, average='binary')
    prc = precision_score(y_test, y_predict, average='binary')
    
    return {
        'FScore': f1, 
        'Roc-Auc': roc,
        'Precision': prc, 
        'Recall': rec,
    }
    
simple_model = evaluate_results(y_test, y_predict)

In [7]:
simple_model

{'FScore': 0.8317757009345793,
 'Roc-Auc': 0.699623745819398,
 'Precision': 0.8090909090909091,
 'Recall': 0.8557692307692307}

Сделаем функцию для полного цикла PU-Learning

In [8]:
def pu_learn(p_prop):
    
    mod_data = df.copy()

    #представим, что нам неизвестны негативы и часть позитивов
    #возьмем индексы положительных положительных экземпляров
    pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]

    #перемешаем их
    np.random.shuffle(pos_ind)

    # оставим необходимое количество положительных экземпляров (сколько мы подаем на вход функции)
    pos_sample_len = int(np.ceil(p_prop * len(pos_ind)))
    pos_sample = pos_ind[:pos_sample_len]
    
    #создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)
    mod_data['class_test'] = -1
    mod_data.loc[pos_sample,'class_test'] = 1

    x_data = mod_data.iloc[:,:-2].values # вся наша выборка
    y_labeled = mod_data.iloc[:,-1].values # новый класс
    y_positive = mod_data.iloc[:,-2].values # оригинальный класс

    #Random Negative Sampling
    mod_data = mod_data.sample(frac=1)
    neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
    sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
    pos_sample = mod_data[mod_data['class_test']==1]
    sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

    model = XGBClassifier(use_label_encoder=False)

    model.fit(sample_train.iloc[:,:-2].values, 
              sample_train.iloc[:,-2].values)
    y_predict = model.predict(sample_test.iloc[:,:-2].values)
    
    return evaluate_results(sample_test.iloc[:,-2].values, y_predict)

In [9]:
pu_01, pu_025, pu_05, pu_07 = pu_learn(0.1), pu_learn(0.25), pu_learn(0.5), pu_learn(0.7)

[17:47:56] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:47:56] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:47:56] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:47:56] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [10]:
result = pd.DataFrame({
    'Without PU': pd.Series(simple_model), 
    'P rate is 0.1': pd.Series(pu_01),
    'P rate is 0.25': pd.Series(pu_025),
    'P rate is 0.5': pd.Series(pu_05),
    'P rate is 0.7':pd.Series(pu_07),
})

result

,Without PU,P rate is 0.1,P rate is 0.25,P rate is 0.5,P rate is 0.7
FScore,0.831776,0.813733,0.796296,0.746736,0.695652
Roc-Auc,0.699624,0.567475,0.606786,0.658749,0.671717
Precision,0.809091,0.703283,0.689840,0.647059,0.571429
Recall,0.855769,0.965338,0.941606,0.882716,0.888889


Количество positive значений, которое мы берем для PU-Learning сильно меняет метрики. Вообще обучение этим методом сильно понизило точность нашей модели, остальные метрики не так сильно, но тоже упали. Возможно это связано с дисбалансом классов, либо с переобучением. Оптимальное значение P - около 0.5. При остальных очень сильно занижаются все метрики.